In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn.functional as F
from PIL import Image

In [2]:
transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
])
train_data = ImageFolder(root="./CarsDataset/train", transform=transform)
test_data = ImageFolder(root="./CarsDataset/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [3]:
class CNN(nn.Module):
    def __init__(self, num_classes, seed = 155):
        super(CNN, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.bn4 = nn.BatchNorm2d(128)
    
        self.fc1 = nn.Linear(10*10*128, 512) 
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x): #el input de cada uno es el input de la conv, por el tamaño de la imagen, y el output lo mismo. El tamaño de la imagen se reduce en kernel-stride
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
    
        x = x.view(x.size(0), -1)
    
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

    def predict_image(self, image_path):
        image = transform(Image.open(image_path)).unsqueeze(0)

        with torch.no_grad():
            output = self(image)

        probabilities = torch.softmax(output, dim=1)[0]
        predicted_label = torch.argmax(probabilities).item()

        class_names = train_data.classes
        predicted_class = class_names[predicted_label]

        print(f'Predicted Class for {image_path}: {predicted_class}')
        for i, prob in enumerate(probabilities):
            print(f'{class_names[i]}: {prob:.4f}')

model = CNN(num_classes=len(train_data.classes))

In [4]:
learning_rate = 0.001 #tested with many, but this one seems the best
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
epochs = 24
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}")

print('Finished Training')

Epoch 1, Training Loss: 1.746369955653236
Epoch 2, Training Loss: 1.4759357724870954
Epoch 3, Training Loss: 1.2855018950643993
Epoch 4, Training Loss: 1.0965463093348913
Epoch 5, Training Loss: 0.9010610202948253
Epoch 6, Training Loss: 0.7187214011237735
Epoch 7, Training Loss: 0.571584529536111
Epoch 8, Training Loss: 0.4017501447172392
Epoch 9, Training Loss: 0.30280913951851074
Epoch 10, Training Loss: 0.2125521719455719
Epoch 11, Training Loss: 0.17461413180544264
Epoch 12, Training Loss: 0.15315540741596903
Epoch 13, Training Loss: 0.12239125074730033
Epoch 14, Training Loss: 0.09930742314706247
Epoch 15, Training Loss: 0.07264547791065915
Epoch 16, Training Loss: 0.07247684808403608
Epoch 17, Training Loss: 0.051697693379329786
Epoch 18, Training Loss: 0.058978976542130114
Epoch 19, Training Loss: 0.07629377153879476
Epoch 20, Training Loss: 0.07411120342245946
Epoch 21, Training Loss: 0.06025229925954981
Epoch 22, Training Loss: 0.03810229376291058
Epoch 23, Training Loss: 0.0

In [6]:
# Evaluation of the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Accuracy on test set: {100 * correct / total}%")

Accuracy on test set: 67.65067650676507%


In [7]:
image_folder = "./CarsDataset/extra"
image_paths = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith(('.jpg', '.jpeg', '.png'))]

for image_path in image_paths:
    model.predict_image(image_path)

Predicted Class for ./CarsDataset/extra\Audi.jpg: Audi
Audi: 1.0000
Hyundai Creta: 0.0000
Mahindra Scorpio: 0.0000
Rolls Royce: 0.0000
Swift: 0.0000
Tata Safari: 0.0000
Toyota Innova: 0.0000
Predicted Class for ./CarsDataset/extra\PruebaMahindraScorpio.jpg: Toyota Innova
Audi: 0.0000
Hyundai Creta: 0.0005
Mahindra Scorpio: 0.0000
Rolls Royce: 0.0000
Swift: 0.0001
Tata Safari: 0.0000
Toyota Innova: 0.9995
Predicted Class for ./CarsDataset/extra\PruebaMahindraScorpio2.png: Audi
Audi: 0.6270
Hyundai Creta: 0.0045
Mahindra Scorpio: 0.0000
Rolls Royce: 0.0000
Swift: 0.0000
Tata Safari: 0.0000
Toyota Innova: 0.3684
Predicted Class for ./CarsDataset/extra\PruebaRollsRoyce.jpg: Rolls Royce
Audi: 0.0000
Hyundai Creta: 0.0000
Mahindra Scorpio: 0.0000
Rolls Royce: 1.0000
Swift: 0.0000
Tata Safari: 0.0000
Toyota Innova: 0.0000


___
SAVE AND LOAD FUNCTIONS
___

In [10]:
# Save the trained model
torch.save({
    'epoch': epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': running_loss/len(train_loader)
}, 'trained_model.pth')

In [12]:
# Load the trained model
model = CNN(num_classes=len(train_data.classes))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

checkpoint = torch.load('trained_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']